# Linear and Polynomial Regression for Cluster Prediction

This notebook uses linear regression and polynomial regression to predict cluster associations from the dataset, after removing foreign key columns.

In [ ]:
# Importations de base

import os

import pandas as pd

import numpy as np

import re

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score

import joblib



# Réglages d'affichage

pd.set_option('display.max_columns', None)

sns.set(style='whitegrid')

In [ ]:
# Chemin vers les données (ajustez si nécessaire)

data_path = os.path.join('Projet', 'clustered_data.csv')

print('Chargement :', data_path)

df = pd.read_csv(data_path)



# Aperçu rapide

print('Taille :', df.shape)

display(df.head())

display(df.dtypes)

display(df.describe(include='all').T)

In [ ]:
# Préparation des données : suppression des colonnes FK et encodage de la cible

# Détection automatique des colonnes FK (heuristique sur les noms)

fk_pattern = re.compile(r'(^id$|_id$|^id_|fk|_fk$|^fk_)', re.I)

fk_cols = [c for c in df.columns if fk_pattern.search(c)]

print('Colonnes FK détectées et exclues :', fk_cols)



# Encodage de la colonne cluster (cible) en numérique pour la régression

cluster_mapping = {cluster: idx for idx, cluster in enumerate(df['cluster'].unique())}

df['cluster_encoded'] = df['cluster'].map(cluster_mapping)

print('Mapping des clusters :', cluster_mapping)



# Colonnes à retirer des features : target + FK détectées

to_drop = ['cluster'] + fk_cols

X = df.drop(columns=to_drop)

y = df['cluster_encoded'].copy()



numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()

categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()



# Préprocesseur : standardisation des numériques, one-hot pour catégorielles

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder



numeric_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(handle_unknown='ignore')



preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder='drop',
)



# Split des données

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



print('Formes après split :')
print('X_train:', X_train.shape, 'y_train:', y_train.shape)
print('X_test:', X_test.shape, 'y_test:', y_test.shape)

In [ ]:
# Régression linéaire

from sklearn.pipeline import Pipeline



linear_pipeline = Pipeline(steps=[('preproc', preprocessor), ('reg', LinearRegression())])



linear_pipeline.fit(X_train, y_train)



# Prédictions

y_train_pred_linear = linear_pipeline.predict(X_train)

y_test_pred_linear = linear_pipeline.predict(X_test)



# Évaluation

train_mse_linear = mean_squared_error(y_train, y_train_pred_linear)

test_mse_linear = mean_squared_error(y_test, y_test_pred_linear)

train_r2_linear = r2_score(y_train, y_train_pred_linear)

test_r2_linear = r2_score(y_test, y_test_pred_linear)



print('=== RÉGRESSION LINÉAIRE ===')
print(f'MSE TRAIN: {train_mse_linear:.4f} | TEST: {test_mse_linear:.4f}')
print(f'R2  TRAIN: {train_r2_linear:.4f} | TEST: {test_r2_linear:.4f}')



# Sauvegarde du modèle

joblib.dump(linear_pipeline, 'linear_regression_cluster.joblib')

print('Modèle linéaire sauvegardé : linear_regression_cluster.joblib')

In [ ]:
# Régression polynomiale (degré 2)

poly_pipeline = Pipeline(steps=[
    ('preproc', preprocessor),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('reg', LinearRegression())
])



poly_pipeline.fit(X_train, y_train)



# Prédictions

y_train_pred_poly = poly_pipeline.predict(X_train)

y_test_pred_poly = poly_pipeline.predict(X_test)



# Évaluation

train_mse_poly = mean_squared_error(y_train, y_train_pred_poly)

test_mse_poly = mean_squared_error(y_test, y_test_pred_poly)

train_r2_poly = r2_score(y_train, y_train_pred_poly)

test_r2_poly = r2_score(y_test, y_test_pred_poly)



print('=== RÉGRESSION POLYNOMIALE (degré 2) ===')
print(f'MSE TRAIN: {train_mse_poly:.4f} | TEST: {test_mse_poly:.4f}')
print(f'R2  TRAIN: {train_r2_poly:.4f} | TEST: {test_r2_poly:.4f}')



# Sauvegarde du modèle

joblib.dump(poly_pipeline, 'polynomial_regression_cluster.joblib')

print('Modèle polynomial sauvegardé : polynomial_regression_cluster.joblib')

In [ ]:
# Visualisations : réel vs prédit pour les deux modèles

# Linéaire

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(y_test, y_test_pred_linear, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Réel (cluster encodé)')
plt.ylabel('Prédit (linéaire)')
plt.title('Réel vs Prédit - Régression Linéaire')

# Polynomial

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred_poly, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Réel (cluster encodé)')
plt.ylabel('Prédit (polynomial)')
plt.title('Réel vs Prédit - Régression Polynomiale')

plt.tight_layout()
plt.show()

## Remarques

- Les clusters ont été encodés numériquement pour permettre la régression.
- Les colonnes FK ont été automatiquement supprimées.
- Pour améliorer les performances, vous pouvez ajuster le degré polynomial ou ajouter de la régularisation.
- Les modèles sont sauvegardés pour une utilisation future.

In [ ]:
# Création du DataFrame de Sortie avec les prédictions et les clés de dimensions
import pandas as pd

# S'assurer que les indices correspondent bien pour une fusion correcte
X_test_reset = X_test.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)

# Création du DataFrame de prédictions
# On copie X_test pour conserver les colonnes de dimensions (features)
predictions_df = X_test_reset.copy()

# Ajout de la colonne avec les vraies valeurs encodées des clusters
predictions_df['cluster_encoded_actual'] = y_test_reset

# Ajout de la colonne avec les prédictions du modèle polynomial
predictions_df['cluster_encoded_predicted_poly'] = y_test_pred_poly

# --- Optionnel : Inverser l'encodage pour avoir les noms de cluster si le mapping est disponible ---
# Pour afficher les noms de cluster réels (si le mapping est nécessaire et disponible globalement)
# if 'cluster_mapping' in locals():
#     reverse_cluster_mapping = {v: k for k, v in cluster_mapping.items()}
#     predictions_df['cluster_actual'] = predictions_df['cluster_encoded_actual'].map(reverse_cluster_mapping)
#     # Pour les prédictions, les valeurs sont des flottants, donc les mapper directement pourrait être trompeur.
#     # On pourrait envisager d'arrondir ou d'utiliser une logique de classification pour les attribuer à un cluster 'nommé'.


# Affichage des premières lignes du DataFrame de prédictions
print("Aperçu du DataFrame de prédictions (premières 5 lignes) :")
display(predictions_df.head())

# Exportation du DataFrame de prédictions vers un fichier CSV
output_filename = 'cluster_predictions_polynomial_regression.csv'
predictions_df.to_csv(output_filename, index=False)
print(f"\nDataFrame de prédictions exporté avec succès sous : {output_filename}")

# Pour une visualisation complète (peut être lourd si beaucoup de données)
# print("\nInformations sur le DataFrame de prédictions :")
# predictions_df.info()
